# Train an LSTM based controller 

Train and save an LSTM-based controller. It contains:
* Code for loading and pre-processing the training data. 
* Training an LSTM with specific parameters and saving it

In [13]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
#from pprint import pformat
from tqdm import tqdm

#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

from sensorprocessing import sp_conv_vae
from demo_to_trainingdata import create_RNN_training_sequence_xy, BCDemonstration
from bc_LSTM import LSTMXYPredictor, LSTMResidualController

from tensorboardX import SummaryWriter


### Creating training and validation data
Create training and validation data from all the demonstrations of a certain task.

In [14]:
task = "proprioception-uncluttered"

conv_vae_jsonfile = pathlib.Path(Config()["controller"]["vae_json"])
conv_vae_model_pthfile = pathlib.Path(Config()["controller"]["vae_model"])


sp = sp_conv_vae.ConvVaeSensorProcessing(conv_vae_jsonfile,
                                         conv_vae_model_pthfile)

demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)

inputlist = []
targetlist = []

for demo_dir in task_dir.iterdir():
    if not demo_dir.is_dir():
        pass
    bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
    print(bcd)
    z, a = bcd.read_z_a()
    print(z.shape)
    print(a.shape)
    # FIXME the repeated name for inputs and targets
    inputs, targets = create_RNN_training_sequence_xy(z, a, sequence_length=10)
    inputlist.append(inputs)
    targetlist.append(targets)

inputs = torch.cat(inputlist)
targets = torch.cat(targetlist)

# Separate the training and validation data. 
# We will be shuffling the demonstrations 
rows = torch.randperm(inputs.size(0)) 
shuffled_inputs = inputs[rows]
shuffled_targets = targets[rows]

training_size = int( inputs.size(0) * 0.67 )
inputs_training = shuffled_inputs[1:training_size]
targets_training = shuffled_targets[1:training_size]

inputs_validation = shuffled_inputs[training_size:]
targets_validation = shuffled_targets[training_size:]

Cameras found: ['dev2']
There are 753 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 753,
 'sensorprocessor': <sensorprocessing.sp_conv_vae.ConvVaeSensorProcessing object at 0x71006e3d3370>,
 'source_dir': PosixPath('/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-demos/demos/proprioception-uncluttered/2024_10_26__16_31_40'),
 'trim_from': 1,
 'trim_to': 753}
(752, 128)
(752, 6)
Cameras found: ['dev2']
There are 968 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 968,
 'sensorprocessor': <sensorprocessing.sp_conv_vae.ConvVaeSensorProcessing object at 0x71006e3d3370>,
 'source_dir': PosixPath('/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-demos/demos/proprioception-uncluttered/2024_10_26__

In [15]:
def validate_behavior_cloning(model, criterion, inputs_validation, targets_validation):
    num_sequences = inputs_validation.shape[0]
    model.eval()
    val_loss = 0
    with torch.no_grad():  # Disable gradient computation
        for i in range(num_sequences):
            # Forward pass
            input_seq = inputs_validation[i]
            target = targets_validation[i]
            # Reshape for batch compatibility
            input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
            target = target.unsqueeze(0)        # Shape: [1, latent_size]

            outputs = model(input_seq)
            loss = criterion(outputs, target)
            # Accumulate loss
            val_loss += loss.item()
    avg_loss = val_loss / num_sequences
    return avg_loss

def train_behavior_cloning(model, optimizer, criterion, inputs_training, targets_training, inputs_validation, targets_validation, num_epochs, writer = None):
    num_sequences = inputs_training.shape[0]

    for epoch in tqdm(range(num_epochs)):
        model.train()
        
        # Loop over each sequence in the batch
        training_loss = 0
        for i in range(num_sequences):
            # Prepare input and target
            input_seq = inputs_training[i]
            target = targets_training[i]

            # Reshape for batch compatibility
            input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
            target = target.unsqueeze(0)        # Shape: [1, latent_size]

            # Forward pass
            output = model(input_seq)
            loss = criterion(output, target)
            training_loss += loss.item()
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        avg_training_loss = training_loss / num_sequences
        avg_validation_loss = validate_behavior_cloning(model, criterion, inputs_validation=inputs_validation, targets_validation=targets_validation)
        if writer is not None:
            writer.add_scalar("TrainingLoss", avg_training_loss, epoch)
            writer.add_scalar("ValidationLoss", avg_validation_loss, epoch)
            writer.flush()


        if (epoch+1) % 2 == 0: # was 0
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_training_loss:.4f} Validation Loss: {avg_validation_loss:.4f} ')




In [ ]:
# Original
latent_size = Config()["robot"]["latent_encoding_size"]  
hidden_size = 32  # degrees of freedom in the robot
output_size = 6  # degrees of freedom in the robot
num_layers = 2

# Instantiate model, loss function, and optimizer
model = LSTMXYPredictor(latent_size=latent_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 100

# Create a SummaryWriter instance
# where does the logdir go???
writer = SummaryWriter(logdir="/home/lboloni/runs/example")

train_behavior_cloning(
    model, optimizer, criterion,
    inputs_training=inputs_training, 
    targets_training=targets_training, 
    inputs_validation=inputs_validation,
    targets_validation=targets_validation,
    num_epochs=num_epochs, writer=writer)
print("Training complete.")
writer.close()


  0%|          | 0/100 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:14<12:09,  7.44s/it]

Epoch [2/100], Training Loss: 681.1389 Validation Loss: 673.9735 


  4%|▍         | 4/100 [00:30<12:33,  7.85s/it]

Epoch [4/100], Training Loss: 657.6181 Validation Loss: 673.2660 


...

 98%|█████████▊| 98/100 [12:05<00:15,  7.53s/it]

Epoch [98/100], Training Loss: 132.4732 Validation Loss: 132.8127 


100%|██████████| 100/100 [12:21<00:00,  7.41s/it]

Epoch [100/100], Training Loss: 135.8327 Validation Loss: 134.6216 
Training complete.


NameError: name 'write' is not defined

In [ ]:

# FIXME: save the model
filename_lstm = Config()["controller"]["lstm_model_file"]
torch.save(model.state_dict(), filename_lstm)

# Load the behavior cloning controller and use it with a real time data

In [ ]:
# Original
latent_size = Config()["robot"]["latent_encoding_size"]  
hidden_size = 32  # degrees of freedom in the robot
output_size = 6  # degrees of freedom in the robot
num_layers = 2

# Instantiate model, loss function, and optimizer
model = LSTMXYPredictor(latent_size=latent_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers)
criterion = nn.MSELoss()  # Mean Squared Error for regression
filename_lstm = Config()["controller"]["lstm_model_file"]
model.load_state_dict(torch.load(filename_lstm))

In [ ]:
# Get one demonstration
task = "proprioception-uncluttered"
sp = sp_conv_vae.ConvVaeSensorProcessing()

demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)

inputlist = []
targetlist = []

demo_dir = next(task_dir.iterdir())
bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
z, a = bcd.read_z_a()

In [ ]:
z.shape[0]
print(a[1])

In [ ]:
for i in range(z.shape[0]-1):
    input = torch.from_numpy(z[i])
    input = input.unsqueeze(0)
    input = input.unsqueeze(0)
    print(input)
    a_pred = model.forward_keep_state(input)
    a_real = a[i+1]
    print(f"a_real: {a_real}\na_pred: {a_pred}")